# Agent配置
## Flexible-Agent
本节目的让Agent真正具备智能与个性，就必须为它设定：
- 它该如何思考（instructions）
- 它依赖的模型是什么（model）
- 它能使用哪些外部工具（tools）

配置 = 定义 Agent 的“性格 + 能力 + 工具箱”。

## 核心配置项

| 配置项         | 说明                                                                 |
| -------------- | -------------------------------------------------------------------- |
| name           | Agent 的名称，用于标识与管理                                         |
| instructions   | 系统提示（System Prompt），定义 Agent 的角色与行为风格                |
| model          | 使用的模型（如 gpt-4o-mini, gpt-4-turbo, gpt-3.5-turbo 等）          |
| tools          | Agent 可调用的工具（如浏览器、Python 代码执行器、文件处理工具）       |
| model_settings | 对模型的参数微调，如温度 temperature、采样率 top_p。                  |
| temperature    | 控制生成的随机性，影响创造性与稳定性                                 |

## 示例拆解--工具使用
天气助手

### 导入pip

```python
from agents import Agent, ModelSettings, function_tool, Runner, set_tracing_disabled
from agents.extensions.models.litellm_model import LitellmModel
import os, asyncio
from dotenv import load_dotenv
```

- Agent：定义智能体
- function_tool：用于装饰自定义函数，让 Agent 能调用
- Runner：执行 Agent
- LitellmModel：包装对 Mistral API 的访问

### 环境与模型配置

```python
load_dotenv()
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)
```

- 从 .env 文件加载 Mistral API Key
- 设置模型为 Mistral Small Latest
- LitellmModel 封装了请求接口
- 关闭追踪（方便本地运行）

### 定义工具函数

```python
@function_tool 
def get_weather(city: str) -> str: 
    """查询一个城市的天气
    Args:
        city (str): 要查询的城市名称
    Returns:
        str: 城市的天气情况
    """
    print(f"天气查询工具被调用了，查询的是{city}")
    return f"The weather in {city} is sunny" 
```

- @function_tool 会把这个函数注册为 Agent 的一个“外部工具”
- 函数的参数和返回值会被自动转换为 JSON Schema，用于模型理解和调用

### 创建Agent

```python
agent = Agent(
    name="天气助手",
    instructions="你是一个专业的天气助手，能够查询不同城市的天气情况。",
    model=llm,
    tools=[get_weather]
)
```

- name：Agent 名称；
- instructions：告诉它回答风格；
- tools：绑定刚定义的 get_weather；
- model：绑定前面构造的 LLM。

### 运行Agent

```python
async def main():
    result = await Runner.run(agent, "上海最近适合出去玩吗?")
    print(result.final_output)
```

## 输出类型（output_type）
默认情况下，Agent输出是字符串类型`str`
但是大部分情况下，我们需要的是结构化数据，如JSON、字典、表格等。

示例：Pydantic对象输出

```python
class Candidate(BaseModel):
    name: str
    gender: str
    location: str
# 定义一个结构化类型
agent = Agent(
    name="简历助手"
    instructions="根据要求的格式提取相应的信息"
    model=llm,
    output_type=Candidate
)
# 执行层
result = await Runner.run(agent,"我叫李婷，女，现居深圳，需要应聘项目经理岗位。")
print(result.final_output)
```

这是一个Pydantic对象，方便后续处理或保存数据库

### 上下文（Context）

LLM本身无状态，通过与设置上下文（Context）来实现状态管理，包含：身份定义、临时变量设定等

上下文是通过RunContextWrapper封装传递给Agent、工具等对象。
其核心作用是：
> "为每次Agent运行提供共享的依赖和状态"

示例：
```python
# 用户实例
@dataclass
class UseInfo:
    name:str
    uid:int

# 定义上下文类

@function_tool
# 工具函数，写一个获取用户ID
async def fetch_user_id(wrapper:RunContextWrapper[UseInfo]) -> str:# ->str为最终返回数据类型
    """
    获取当前用户的ID信息。

    Args:
        wrapper (RunContextWrapper[UseInfo]): 包含用户上下文信息的包装器，
            可通过 wrapper.context 访问 UseInfo 实例。

    Returns:
        str: 格式化的用户ID字符串，例如 "User John's ID is 123"。
    """ 
    return f"用户{wrapper.context.name}的ID是{wrapper.context.uid}"
```
- 工具函数通过wrapper.context访问当前用户
- 不同Agent、工具之间共享相同的上下文类系

```python
async def main():
    user_info = UserInfo(name="John", uid=123)

    agent = Agent[UserInfo](  
        name="Assistant",
        tools=[fetch_user_age],
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="What is the ID of the user?",
        context=user_info,
    )

    print(result.final_output) 
```
- context=user_info ：将user_info传递给Agent，传递上下文作用
- 输出：The user John's ID is 123.
- `agent = Agent[UseInfo]`，其中UseInfo表示您可以通过wrapper.context访问的上下文对象类型。

>上下文不会发送给LLM，而是本地的依赖容器。
对应到Dify中的“上下文变量”而非“全局变量”

### 动态指令（Dynamic Instructions）

有时指令不是固定的，需要根据上下文动态生成。如：通过函数调用获取用户信息，根据用户信息动态生成指令。实现了软编码。

```python
def dynamic_instructions(
    context: RunContextWrapper[UseInfo], agent: Agent[UseInfo]
) -> str:
    return f"The user's name is {context.context.name}. Help them with their questions."


agent = Agent[UseInfo](
    name="Triage agent",
    instructions=dynamic_instructions,
)
```
实例演示：
```python
import asyncio
from dataclasses import dataclass
from agents.extensions.models.litellm_model import LitellmModel
from agents import Agent, RunContextWrapper, Runner, function_tool, set_tracing_disabled
import os
import asyncio
from dotenv import load_dotenv

# 加载环境变量
load_dotenv()
# 从环境变量中读取api_key
api_key = os.getenv('mistral_key')
base_url = 'https://api.mistral.ai/v1'
chat_model = "mistral/mistral-small-latest"
set_tracing_disabled(disabled=True)
llm = LitellmModel(model=chat_model, api_key=api_key, base_url=base_url)


@dataclass
class PatInfo:
    ip_type: str
    def advice(self) -> str:
        if self.ip_type=="发明":
            return "重点为用户讲解发明专利申请的实质审查要求"
        elif self.ip_type=="实用新型":
            return "重点为用户讲解实用新型专利申请的形式审查要求"

def dynamic_instructions(
    context: RunContextWrapper[PatInfo], agent: Agent[PatInfo]
) -> str:
    return f"用汉赋的句式{context.context.advice()}."


async def main():
    ip_info = PatInfo(ip_type="实用新型")

    agent = Agent[PatInfo](  
        name="Assistant",
        instructions=dynamic_instructions,
        model=llm,
    )

    result = await Runner.run(  
        starting_agent=agent,
        input="我想申请专利",
        context=ip_info,
    )

    print(result.final_output)  

if __name__ == "__main__":
    asyncio.run(main())
```
- Agent[PatInfo]：表示Agent可以访问PatInfo数据类。
- 传入context=ip_info ：将ip_info传递给Agent，传递上下文作用
- agent中的dynamic_instructions函数使用了函数PatInfo.advice()去构建了**动态指令**。
- 动态指令输出：用汉赋的句式重点为用户讲解实用新型专利申请的形式审查要求。该字段作为指令传入agent。

动态指令在Dify中直接指向了其System Prompt。使用了dynamic_instructions()。
参考下图理解。
```
[PatInfo 数据类]  ──▶  [dynamic_instructions 函数]
        ↓                          ↓
       生成新的 system prompt  →  注入 Agent
                              ↓
               用户输入 prompt 合并执行
                              ↓
                     LLM 生成最终输出
```
举个例子：

| 用户类型 | 动态指令输出                                   |
|----------|------------------------------------------------|
| VIP 用户 | “你是一名温柔的VIP客服，请使用尊称‘您’。”       |
| 普通用户 | “你是一名普通客服，请使用友好口吻。”           |

### 克隆（Clone）
通过克隆可以创建一个新的Agent，新Agent的指令与原Agent相同，但是可以有不同的工具集。
```python
pirate_agent = Agent(
    name="Pirate",
    instructions="Write like a pirate",
    model="o3-mini",
)

robot_agent = pirate_agent.clone(
    name="Robot",
    instructions="Write like a robot",
)
```

## 总结

| 配置项               | 类别               | 说明                                                                 |
| -------------------- | ------------------ | -------------------------------------------------------------------- |
| instructions         | 行为定义           | 定义 Agent 行为，示例：“用汉赋风格回答”                               |
| model                | 模型指定           | 指定 LLM，可选项包括 Mistral、小模型或 OpenAI 系列模型                |
| tools                | 工具注册           | 注册外部函数，示例：`@function_tool`                                 |
| output_type          | 输出格式           | 结构化输出，基于 BaseModel 类实现                                    |
| context              | 状态管理           | 本地状态共享，通过 RunContextWrapper 实现                             |
| clone()              |  Agent 复制        | 快速复制 Agent，示例：“Robot from Pirate”                            |
| dynamic_instructions | 提示生成           | 动态生成提示，可根据上下文变化调整提示内容                           |

## 拓展思考

LLM什么时候决定调用Tool？
- Context/Dynamic Instructions中包含了函数调用的指令时
- 当llm知识不够
- 当用户请求执行
- 输入语义与工具描述高度匹配
- 需要结构化输出